#### Place cell remapping

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.ndimage import gaussian_filter
from sklearn.feature_selection import mutual_info_regression as mi_skl
import seaborn as sns
from scipy import stats
import time
from matplotlib import gridspec
import os
from scipy.spatial import distance

#Load data
ANIMALS = []
#-------------------------------------------------------------------------------------------

#File paths - adapt to local situation
codeDir = r'C:\Users\fr87_\OneDrive - University of Edinburgh\arena room files\ANALYSIS 2.0\Code\Gobbo et al. Code' #os.getcwd() 
#len2del = sum([len(i) for i in codeDir.split('/')[-2:]])+1
homeDir = codeDir#[:-len2del]
BEHdir = homeDir + '\\Example Data\\%s\\%s\\BEH' %(Animal_ID, Ego_Allo)
EVEdir = homeDir + '\\Example Data\\%s\\%s\\ALIGNED TRA EVE\\REGISTERED\\Events\\Quality_checked'  %(Animal_ID, Ego_Allo)
TRImet = homeDir + '\\Example Data\\Trial_META_PRE.csv' #this contains the start and end times of PRE (exploration) files. It is located in Example Data

homeDir = codeDir[:-len2del]
dataDir = codeDir[:-sum([len(i) for i in codeDir.split('/')[-3:]])-2] + 'animal_data/'
SaveTo2 = homeDir+'Figures/Figure 2 Calcium imaging/Panel_E_Centroid_remapping/' #Where to save figures

pcDIR = homeDir + "Data/Place cell meta/" # where place cell table is saved

#Split up the arena <<-------- adapt to new version
def arena_binned(x, y, xdim, ydim, pix_cm, pf_cm):
    d_x = xdim[0] - xdim[1] # xdim[0] = max, xdim[1] = min
    d_y = ydim[0] - ydim[1] # ydim[0] = max, ydim[1] = min
    nBnx = int((d_x/(pix_cm*pf_cm)))
    nBny = int((d_y/(pix_cm*pf_cm)))
    bn_x = [int(i) for i in np.linspace(xdim[0],xdim[1],nBnx)]
    bn_y = [int(i) for i in np.linspace(ydim[0], ydim[1],nBny)]

    bx = ((x-bn_x[0])/(bn_x[1]-bn_x[0])).astype(int)
    by = ((y-bn_y[0])/(bn_y[1]-bn_y[0])).astype(int)
#     bx[bx>=20] = 19
#     by[by>=20] = 19
    S = np.vstack((bx,by))
    print(np.max(S))
    linS = np.ravel_multi_index(S,(nBnx,nBny))
    occMap = sps.csr_matrix((np.ones(len(bx)),(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
    return linS.astype(float), occMap, nBnx, nBny, bx, by

#ADDITIONAL PARAMS <<-------- ADAPT FOR NEW ARENA
xdim = [350, -10] # Set arena x dimensions
ydim = [400,-10] # Set Arena y dimensions
pix_cm = 5 # Set how many pixels = 1 cm
pf_cm = 4 # Set bin size for place field (cms)
occplot = False
cell_type = 1

#### Calculates centroid distance between stages EXP, SAM, CHO (there is also code only or EXP across sessions below)

In [ ]:
Centroid_dis = pd.DataFrame({"Animal":[], "Session":[], "Stage":[], "Cell":[],
                             "Displacement x (cm)":[], "Displacement y (cm)":[], "Displacement absolute (cm)":[]})
for animal in ANIMALS:
    
    #Read in PC data table
    META_PC = pd.read_csv(pcDIR + '%s_place_cell_meta.csv' %animal)
    animDIR = dataDir + str(animal) + '/01_Aligned_LR/'

    for session in aniSESSIONS[animal]:
        
        #Read in animal, session, stage meta
        EXP = pd.read_csv(animDIR +'/%s_%s_PRE_event_dlc_LR.csv'%(animal,session))
        SAM = pd.read_csv(animDIR +'/%s_%s_SAM_event_dlc_LR.csv'%(animal,session))
        CHO = pd.read_csv(animDIR +'/%s_%s_CHO_event_dlc_LR.csv'%(animal,session))
        
        #Extract x,y actvity 
        EXP_x = EXP['x'].values; EXP_y = EXP['y'].values 
        SAM_x = SAM['x'].values; SAM_y = SAM['y'].values 
        CHO_x = CHO['x'].values; CHO_y = CHO['y'].values 
        
        #Bin and linearize occupancy
        ElinS, EoccMap, EnBnx, EnBny, Ebx, Eby = arena_binned(x = EXP_x, y = EXP_y, xdim = xdim, ydim = ydim, pix_cm = pix_cm, pf_cm = pf_cm)
        SlinS, SoccMap, SnBnx, SnBny, Sbx, Sby = arena_binned(x = SAM_x, y = SAM_y, xdim = xdim, ydim = ydim, pix_cm = pix_cm, pf_cm = pf_cm)
        ClinS, CoccMap, CnBnx, CnBny, Cbx, Cby = arena_binned(x = CHO_x, y = CHO_y, xdim = xdim, ydim = ydim, pix_cm = pix_cm, pf_cm = pf_cm)
        
        Eocc_nan = np.argwhere(EoccMap == 0)
        Socc_nan = np.argwhere(SoccMap == 0)
        Cocc_nan = np.argwhere(CoccMap == 0)
        
        #Idxs where animal is stationary
        Esta = np.array(EXP.loc[(EXP['Movement status']=='stationary') & (EXP['position']!=12)].index)
        Ssta = np.array(SAM.loc[(SAM['Movement status']=='stationary') & (EXP['position']!=12)].index)
        Csta = np.array(CHO.loc[(CHO['Movement status']=='stationary') & (EXP['position']!=12)].index)
        Emov = np.array(EXP.loc[(EXP['Movement status']=='moving') & (EXP['position']!=12)].index)
        Smov = np.array(SAM.loc[(SAM['Movement status']=='moving') & (SAM['position']!=12)].index)
        Cmov = np.array(CHO.loc[(CHO['Movement status']=='moving') & (CHO['position']!=12)].index)
        
        #Isolate cells that fire in all stages
        EXP_cells = EXP[[i for i in EXP.columns if i.startswith(' C')]]
        EXP_cells = [i for i in EXP_cells.columns if sum(EXP_cells[i][Emov]) > 0]
        SAM_cells = SAM[[i for i in SAM.columns if i.startswith(' C')]]
        SAM_cells = [i for i in SAM_cells.columns if sum(SAM_cells[i][Smov]) > 0]
        CHO_cells = CHO[[i for i in CHO.columns if i.startswith(' C')]]
        CHO_cells = [i for i in CHO_cells.columns if sum(CHO_cells[i][Cmov]) > 0]
        SAMcCells = list(set(EXP_cells).intersection(SAM_cells))
        CHOcCells = list(set(EXP_cells).intersection(CHO_cells))
        cCells = list(set(EXP_cells).intersection(SAM_cells, CHO_cells))

        #Isolate Place cells
        PCs = META_PC.loc[(META_PC['Session'] == session) &
                          (META_PC['Stage'] == 'PRE') & 
                          (META_PC['Place cells'] == cell_type) & 
                          (META_PC['Cell IDS'].isin(SAMcCells+CHOcCells+cCells))]

        for cell in PCs['Cell IDS']:
            #Isolate events per stage
            EXP_ev = EXP[cell].values; EXP_ev[Esta]=0
            SAM_ev = SAM[cell].values; SAM_ev[Ssta]=0
            CHO_ev = CHO[cell].values; CHO_ev[Csta]=0
            
            # OPTIONAL: Binarized event
            EXP_ev[EXP_ev > 0] = 1 
            SAM_ev[SAM_ev > 0] = 1 
            CHO_ev[CHO_ev > 0] = 1 
    
            #Calculate rate and place map for EXP
            ErateMap = sps.csr_matrix((EXP_ev,(Ebx,Eby)),shape=(EnBnx,EnBny),dtype=float).todense()
            EplaceMap = np.divide(ErateMap,EoccMap); EplaceMap[np.isnan(EplaceMap)] = 0
            Epc_plot1 = gaussian_filter(np.array(EplaceMap), sigma = 1)
            Emax_pc_plot = np.max(Epc_plot1)
            Epc_plot = Epc_plot1/Emax_pc_plot
            Epc_plot[Epc_plot <=0.2] = 0
            
            #Calculate rate and place map for SAM
            SrateMap = sps.csr_matrix((SAM_ev,(Sbx,Sby)),shape=(SnBnx,SnBny),dtype=float).todense()
            SplaceMap = np.divide(SrateMap,SoccMap); SplaceMap[np.isnan(SplaceMap)] = 0
            Spc_plot1 = gaussian_filter(np.array(SplaceMap), sigma = 1)
            Smax_pc_plot = np.max(Spc_plot1)
            Spc_plot = Spc_plot1/Smax_pc_plot
            
            #Calculate rate and place map for CHO
            CrateMap = sps.csr_matrix((CHO_ev,(Cbx,Cby)),shape=(CnBnx,CnBny),dtype=float).todense()
            CplaceMap = np.divide(CrateMap,CoccMap); CplaceMap[np.isnan(CplaceMap)] = 0
            Cpc_plot1 = gaussian_filter(np.array(CplaceMap), sigma = 1)
            Cmax_pc_plot = np.max(Cpc_plot1)
            Cpc_plot = Cpc_plot1/Cmax_pc_plot
            
            #Weighted Average Centroid 
            (X,Y) = np.meshgrid(range(0, Epc_plot.shape[0]),range(0, Epc_plot.shape[1]))
            ECx = (X*Epc_plot.T).sum() / Epc_plot.T.sum().astype("float")
            ECy = (Y*Epc_plot.T).sum() / Epc_plot.T.sum().astype("float")
            (X,Y) = np.meshgrid(range(0, Spc_plot.shape[0]),range(0, Spc_plot.shape[1]))
            SCx = (X*Spc_plot.T).sum() / Spc_plot.T.sum().astype("float")
            SCy = (Y*Spc_plot.T).sum() / Spc_plot.T.sum().astype("float")
            (X,Y) = np.meshgrid(range(0, Epc_plot.shape[0]),range(0, Epc_plot.shape[1]))
            CCx = (X*Cpc_plot.T).sum() / Cpc_plot.T.sum().astype("float")
            CCy = (Y*Cpc_plot.T).sum() / Cpc_plot.T.sum().astype("float")
            
            if np.sum(Epc_plot[:,int(ECx-1):int(ECx+1)][int(ECy-1):int(ECy+1)]) > 0.25:
    #             #Max Centroid coordinates
    #             ECidx = np.argmax(Epc_plot); ECy = ECidx%20; ECx = int(ECidx/20) 
    #             SCidx = np.argmax(Spc_plot1); SCy = SCidx%20; SCx = int(SCidx/20) 
    #             CCidx = np.argmax(Cpc_plot1); CCy = CCidx%20; CCx = int(CCidx/20) 

                #Centroid distances x, y (cm) & Euclidean
                if SCx > 0:
                    EC_SCx = (ECx - SCx)*pf_cm; EC_SCy = (ECy - SCy)*pf_cm*-1
                    EC_SC = distance.euclidean([ECx, ECy], [SCx, SCy])*pf_cm
                else:
                    EC_SCx = np.nan; EC_SCy = np.nan; EC_SC = np.nan
                if CCx > 0:
                    EC_CCx = (ECx - CCx)*pf_cm; EC_CCy = (ECy - CCy)*pf_cm*-1
                    EC_CC = distance.euclidean([ECx, ECy], [CCx, CCy])*pf_cm
                else:
                    EC_CCx = np.nan; EC_CCy = np.nan; EC_CC = np.nan

                Centroid_dis = Centroid_dis.append(pd.DataFrame({"Animal":[animal]*2, 
                                                                 "Session":[session]*2,
                                                                 "Stage":['SAM', 'CHO'],
                                                                 "Cell":[cell]*2,
                                                                 "Displacement x (cm)":[EC_SCx, EC_CCx], 
                                                                 "Displacement y (cm)":[EC_SCy, EC_CCy], 
                                                                 "Displacement absolute (cm)":[EC_SC, EC_CC]}))
                if occplot:
                    n_stages = len([i for i in [ECx, SCx, CCx] if i > 0])
                    #Plot stages
                    fig = plt.figure(figsize=(8,3))
                    if n_stages == 2:
                        fig = plt.figure(figsize=(4,3))
                        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 3])
                    else:
                        fig = plt.figure(figsize=(8,3))
                        gs = gridspec.GridSpec(1, 3, width_ratios=[3, 3, 3])
                    fig.suptitle('%s %s %s'%(animal, session, cell), fontsize=16, y=1.2,)
                    fig.tight_layout() 

                    gs_idx = 0
                    plt.subplot(gs[gs_idx])
                    Epc_plot[np.ix_(Eocc_nan[:,0]),Eocc_nan[:,1]] = np.nan
                    plt.axvline(x=ECx,color='k', linestyle = '--', linewidth=0.5)
                    plt.axhline(y=ECy,color='k', linestyle = '--', linewidth=0.5)
                    plt.imshow(Epc_plot.T, aspect="auto", cmap='viridis')
                    plt.gca().invert_xaxis()
                    plt.gca().invert_yaxis()
                    sns.despine()
                    plt.axis('off')
                    plt.title('EXP\n[max = %.3f]'%(Emax_pc_plot))

                    if SCx > 0:
                        gs_idx += 1
                        plt.subplot(gs[gs_idx])
                        Spc_plot[np.ix_(Socc_nan[:,0]),Socc_nan[:,1]] = np.nan
                        plt.axvline(x=SCx,color='k', linestyle = '--', linewidth=0.5)
                        plt.axhline(y=SCy,color='k', linestyle = '--', linewidth=0.5)
                        plt.imshow(Spc_plot.T, aspect="auto")
                        plt.gca().invert_xaxis()
                        plt.gca().invert_yaxis()
                        sns.despine()
                        plt.axis('off')
                        plt.title('SAM\n[max = %.3f]'%(Smax_pc_plot))

                    if CCx > 0:
                        gs_idx += 1
                        plt.subplot(gs[gs_idx])
                        Cpc_plot[np.ix_(Cocc_nan[:,0]),Cocc_nan[:,1]] = np.nan
                        plt.axvline(x=CCx,color='k', linestyle = '--', linewidth=0.5)
                        plt.axhline(y=CCy,color='k', linestyle = '--', linewidth=0.5)
                        plt.imshow(Cpc_plot.T, aspect="auto")
                        plt.gca().invert_xaxis()
                        plt.gca().invert_yaxis()
                        sns.despine()
                        plt.axis('off')
                        plt.title('CHO\n[max = %.3f]'%(Cmax_pc_plot))
                        plt.savefig(SaveTo2 + 'PRESAMCHO_occplot/Stage/%s/%s_%s_stage_occ_maps.eps' %(animal, animal, cell), dpi=300, bbox_inches="tight")
                        plt.show()


#### Plots polar plots

In [ ]:
if cell_type == 1:
    Saveas = 'PCs'
    cell_color = 'blue'
else:
    Saveas = 'nPCs'
    cell_color = 'gray'

SAM_Centroid_dis = Centroid_dis[Centroid_dis['Stage'] == 'SAM']
CHO_Centroid_dis = Centroid_dis[Centroid_dis['Stage'] == 'CHO']

#Plot EXP vs SAM and EXP vs CHO pooled & same color
ax = sns.jointplot(x='Displacement x (cm)',y='Displacement y (cm)',data=Centroid_dis, color='blue', alpha=0.4, s=10, zorder=0);
ax.plot_joint(sns.kdeplot,  zorder=1);
ax.ax_joint.axvline(x=0, linestyle = '--', linewidth=0.5, c='k')
ax.ax_joint.axhline(y=0, linestyle = '--', linewidth=0.5, c='k')
plt.savefig(SaveTo2 + '/Stage/%s_Centroid_StageRemap_distribution.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

#Plot EXP vs SAM and EXP vs CHO pooled & different colors
ax = sns.jointplot(x='Displacement x (cm)',y='Displacement y (cm)',data=Centroid_dis, hue='Stage', color='blue', alpha=0.4, s=10, zorder=0, palette=[color_dic['SAM'], color_dic['CHO']]);
ax.plot_joint(sns.kdeplot,  zorder=1);
ax.ax_joint.axvline(x=0, linestyle = '--', linewidth=0.5, c='k')
ax.ax_joint.axhline(y=0, linestyle = '--', linewidth=0.5, c='k')
plt.savefig(SaveTo2 + '/Stage/%s_Centroid_StageRemap_distribution_SAMCHO.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

ax = sns.jointplot(x=SAM_Centroid_dis['Displacement x (cm)'],y=SAM_Centroid_dis['Displacement y (cm)'],color=color_dic['SAM'], alpha=0.4, s=10, zorder=0);
ax.plot_joint(sns.kdeplot,  color=color_dic['SAM'], zorder=1);
ax.ax_joint.axvline(x=0, linestyle = '--', linewidth=0.5, c='k')
ax.ax_joint.axhline(y=0, linestyle = '--', linewidth=0.5, c='k')
plt.savefig(SaveTo2 + '/Stage/%s_Centroid_StageRemap_distribution_SAM.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

ax = sns.jointplot(x=CHO_Centroid_dis['Displacement x (cm)'],y=CHO_Centroid_dis['Displacement y (cm)'],color=color_dic['CHO'], alpha=0.4, s=10, zorder=0);
ax.plot_joint(sns.kdeplot, color=color_dic['CHO'], zorder=1);
ax.ax_joint.axvline(x=0, linestyle = '--', linewidth=0.5, c='k')
ax.ax_joint.axhline(y=0, linestyle = '--', linewidth=0.5, c='k')
plt.savefig(SaveTo2 + '/Stage/%s_Centroid_StageRemap_distribution_CHO.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

#### Plots cumulative distribution

In [ ]:
SAM_Cd = Centroid_dis[Centroid_dis['Stage']=='SAM']['Displacement absolute (cm)']
CHO_Cd = Centroid_dis[Centroid_dis['Stage']=='CHO']['Displacement absolute (cm)']

fig, ax = plt.subplots(figsize=(4, 4),dpi=200)
# plot the cumulative histogram
n, bins, patches = ax.hist(SAM_Cd, bins=len(SAM_Cd), density=True, histtype='step',cumulative=True, color=color_dic['SAM'])
n, bins, patches = ax.hist(CHO_Cd, bins=len(CHO_Cd), density=True, histtype='step',cumulative=True, color=color_dic['CHO'])

plt.xlim(0,60)
plt.xlabel('Place field displacement (cm)')
plt.ylabel('Fraction of PCs')
plt.legend(['SAM', 'CHO'], loc='lower right')
plt.savefig(SaveTo2 + 'Stage/%s_Centroid_StageRemap_cumulativedis.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

#### Centroid distance for EXP accross sessions

In [ ]:
occplot = True
Centroid_dis = pd.DataFrame({"Animal":[], "Session":[],"Displacement x (cm)":[], "Displacement y (cm)":[], "Displacement absolute (cm)":[]})
for animal in ANIMALS:
    
    #Read in PC data table
    META_PC = pd.read_csv(pcDIR + '%s_place_cell_meta.csv' %animal)
    animDIR = dataDir + str(animal) + '/01_Aligned_LR/'
    
    #Read in animal, session, stage meta <----- UPATE with all sessions
    A01 = pd.read_csv(animDIR +'/%s_A01_PRE_event_dlc_LR.csv'%(animal))
    A02 = pd.read_csv(animDIR +'/%s_A02_PRE_event_dlc_LR.csv'%(animal))


    #Extract x,y actvity 
    A01_x = A01['x'].values; A01_y = A01['y'].values 
    A02_x = A02['x'].values; A02_y = A02['y'].values 
    
    #Bin and linearize occupancy
    A01linS, A01occMap, A01nBnx, A01nBny, A01bx, A01by = arena_binned(x = A01_x, y = A01_y, xdim = xdim, ydim = ydim, pix_cm = pix_cm, pf_cm = pf_cm)
    A02linS, A02occMap, A02nBnx, A02nBny, A02bx, A02by = arena_binned(x = A02_x, y = A02_y, xdim = xdim, ydim = ydim, pix_cm = pix_cm, pf_cm = pf_cm)
                      
    A01occ_nan = np.argwhere(A01occMap == 0)
    A02occ_nan = np.argwhere(A02occMap == 0)
    Sigocc_nan = np.argwhere((A01occMap+A02occMap) == 0)
        
    #Idxs where animal is stationary & moving
    A01sta = np.array(A01.loc[(A01['Movement status']=='stationary') & (A01['position']!=12)].index)
    A02sta = np.array(A02.loc[(A02['Movement status']=='stationary') & (A02['position']!=12)].index)
    A01mov = np.array(A01.loc[(A01['Movement status']=='moving') & (A01['position']!=12)].index)
    A02mov = np.array(A02.loc[(A02['Movement status']=='moving') & (A02['position']!=12)].index)

    #Isolate cells that fire in all stages
    A01_cells = A01[[i for i in A01.columns if i.startswith(' C')]]
    A01_cells = [i for i in A01_cells.columns if sum(A01_cells[i][A01mov]) > 0]
    A02_cells = A02[[i for i in A02.columns if i.startswith(' C')]]
    A02_cells = [i for i in A02_cells.columns if sum(A02_cells[i][A02mov]) > 0]
    cCells = list(set(A01_cells).intersection(A02_cells))

    #Isolate Place cells
    PCs = META_PC.loc[(META_PC['Stage'] == 'PRE') & 
                      (META_PC['Place cells'] == cell_type) & 
                      (META_PC['Cell IDS'].isin(cCells))]

    for cell in PCs['Cell IDS']:
        #Isolate events per stage
        A01_ev = A01[cell].values; A01_ev[A01sta]=0
        A02_ev = A02[cell].values; A02_ev[A02sta]=0 
        
        # OPTIONAL: Binarized event
        A01_ev[A01_ev > 0] = 1 
        A02_ev[A02_ev > 0] = 1 
    
        #Calculate rate and place map for A01
        A01rateMap = sps.csr_matrix((A01_ev,(A01bx,A01by)),shape=(A01nBnx,A01nBny),dtype=float).todense()
        A01placeMap = np.divide(A01rateMap,A01occMap); A01placeMap[np.isnan(A01placeMap)] = 0
        A01pc_plot1 = gaussian_filter(np.array(A01placeMap), sigma = 1)
        A01max_pc_plot = np.max(A01pc_plot1)
        A01pc_plot = A01pc_plot1/A01max_pc_plot

        #Calculate rate and place map for A02
        A02rateMap = sps.csr_matrix((A02_ev,(A02bx,A02by)),shape=(A02nBnx,A02nBny),dtype=float).todense()
        A02placeMap = np.divide(A02rateMap,A02occMap); A02placeMap[np.isnan(A02placeMap)] = 0
        A02pc_plot1 = gaussian_filter(np.array(A02placeMap), sigma = 1)
        A02max_pc_plot = np.max(A02pc_plot1)
        A02pc_plot = A02pc_plot1/A02max_pc_plot
        
        Sigpc_plot = A01pc_plot + A02pc_plot

        #Weighted Average Centroid 
        (X,Y) = np.meshgrid(range(0, A01pc_plot.shape[0]),range(0, A01pc_plot.shape[1]))
        A01Cx = (X*A01pc_plot.T).sum() / A01pc_plot.T.sum().astype("float")
        A01Cy = (Y*A01pc_plot.T).sum() / A01pc_plot.T.sum().astype("float")
        (X,Y) = np.meshgrid(range(0, A02pc_plot.shape[0]),range(0, A02pc_plot.shape[1]))
        A02Cx = (X*A02pc_plot.T).sum() / A02pc_plot.T.sum().astype("float")
        A02Cy = (Y*A02pc_plot.T).sum() / A02pc_plot.T.sum().astype("float")
        (X,Y) = np.meshgrid(range(0, Sigpc_plot.shape[0]),range(0, Sigpc_plot.shape[1]))
        SigCx = (X*Sigpc_plot.T).sum() / Sigpc_plot.T.sum().astype("float")
        SigCy = (Y*Sigpc_plot.T).sum() / Sigpc_plot.T.sum().astype("float")
        
        #Centroid distance of A01-P02 from avg x,y (cm)
        SigC_A01Cx = (SigCx - A01Cx)*pf_cm; SigC_A01Cy = (SigCy - A01Cy)*pf_cm*-1
        SigC_A02Cx = (SigCx - A02Cx)*pf_cm; SigC_A02Cy = (SigCy - A02Cy)*pf_cm*-1
        C_disx = [SigC_A01Cx, SigC_A02Cx]; C_disy = [SigC_A01Cy, SigC_A02Cy]
        
#         #Centroid coordinates (max)
#         A01Cidx = np.argmax(A01pc_plot1); A01Cy = A01Cidx%20; A01Cx = int(A01Cidx/20) 
#         A02Cidx = np.argmax(A02pc_plot1); A02Cy = A02Cidx%20; A02Cx = int(A02Cidx/20) 
 
#         #Centroid distance x, y (cm)
#         A01C_A02Cx = (A01Cx - A02Cx)*pf_cm; A01C_A02Cy = (A01Cy - A02Cy)*pf_cm*-1
#         C_disx = [A01C_A02Cx]; C_disy = [A01C_A02Cy]

            
        #Euclidean distance (cm)
        SigC_A01C = distance.euclidean([SigCx, SigCy], [A01Cx, A01Cy])*pf_cm
        SigC_A02C = distance.euclidean([SigCx, SigCy], [A02Cx, A02Cy])*pf_cm
        C_dis = [SigC_A01C, SigC_A02C]
        
        session_labels = ['A01', 'A02'][:len(C_dis)]
        Centroid_dis = Centroid_dis.append(pd.DataFrame({"Animal":[animal]*len(C_dis), 
                                                        "Session":session_labels,
                                                        "Displacement x (cm)":C_disx, 
                                                        "Displacement y (cm)":C_disy, 
                                                        "Displacement absolute (cm)":C_dis}))
        if occplot:
            #Plot stages
            fig = plt.figure(figsize=(10,3))
            fig.tight_layout() 
            gs = gridspec.GridSpec(1,3, width_ratios=[3, 3, 3])
            fig.suptitle('%s %s'%(animal, cell), fontsize=16, y=1.08,)
            
            plt.subplot(gs[0])
            Sigpc_plot[np.ix_(Sigocc_nan[:,0]),Sigocc_nan[:,1]] = np.nan
            plt.axvline(x=SigCx,color='k', linestyle = '--', linewidth=0.5)
            plt.axhline(y=SigCy,color='k', linestyle = '--', linewidth=0.5)
            plt.imshow(Sigpc_plot.T, aspect="auto")
            plt.gca().invert_xaxis()
            plt.gca().invert_yaxis()
            sns.despine()
            plt.axis('off')
            plt.title('[max = %.3f]'%(A01max_pc_plot))
            
            plt.subplot(gs[1])
            A01pc_plot[np.ix_(A01occ_nan[:,0]),A01occ_nan[:,1]] = np.nan
            plt.axvline(x=A01Cx,color='k', linestyle = '--', linewidth=0.5)
            plt.axhline(y=A01Cy,color='k', linestyle = '--', linewidth=0.5)
            plt.imshow(A01pc_plot.T, aspect="auto")
            plt.gca().invert_xaxis()
            plt.gca().invert_yaxis()
            sns.despine()
            plt.axis('off')
            plt.title('[max = %.3f]'%(A01max_pc_plot))

            plt.subplot(gs[2])
            A02pc_plot[np.ix_(A02occ_nan[:,0]),A02occ_nan[:,1]] = np.nan
            plt.axvline(x=A02Cx,color='k', linestyle = '--', linewidth=0.5)
            plt.axhline(y=A02Cy,color='k', linestyle = '--', linewidth=0.5)
            plt.imshow(A02pc_plot.T, aspect="auto")
            plt.gca().invert_xaxis()
            plt.gca().invert_yaxis()
            sns.despine()
            plt.axis('off')
            plt.title('[max = %.3f]'%(A02max_pc_plot))

            plt.savefig(SaveTo2 + 'Session/EXPsession_occplot/%s/%s_%s_stage_occ_maps.svg' %(animal, animal, cell), dpi=300, bbox_inches="tight", transparent=True)
            plt.show()


#### Plots polar plots

In [ ]:
if cell_type == 1:
    Saveas = 'PCs'
    cell_color = 'blue'
else:
    Saveas = 'nPCs'
    cell_color = 'gray'
    
ax = sns.jointplot(x=Centroid_dis['Displacement x (cm)'],y=Centroid_dis['Displacement y (cm)'],
                   s=10, color=cell_color, marginal_kws=dict(bins=25, rug=True),alpha=0.2, zorder=0);
ax.plot_joint(sns.kdeplot, color=cell_color,  zorder=1);
ax.ax_joint.axvline(x=0, linestyle = '--', linewidth=0.5, c='k')
ax.ax_joint.axhline(y=0, linestyle = '--', linewidth=0.5, c='k')
plt.savefig(SaveTo2 + 'Session/%s_Centroid_SessionRemap_dis.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()

#### Plots cumulative distribution

In [ ]:
A01_Cd = Centroid_dis[Centroid_dis['Session']=='A01']['Displacement absolute (cm)']
A02_Cd = Centroid_dis[Centroid_dis['Session']=='A02']['Displacement absolute (cm)']
P01_Cd = Centroid_dis[Centroid_dis['Session']=='P01']['Displacement absolute (cm)']
P02_Cd = Centroid_dis[Centroid_dis['Session']=='P02']['Displacement absolute (cm)']

fig, ax = plt.subplots(figsize=(4, 4),dpi=200)
# plot the cumulative histogram
n, bins, patches = ax.hist(A01_Cd, bins=len(A01_Cd), density=True, histtype='step',cumulative=True, color=color_dic['A01'])
n, bins, patches = ax.hist(A02_Cd, bins=len(A02_Cd), density=True, histtype='step',cumulative=True, color=color_dic['A02'])
n, bins, patches = ax.hist(P01_Cd, bins=len(P01_Cd), density=True, histtype='step',cumulative=True, color=color_dic['P01'])
n, bins, patches = ax.hist(P02_Cd, bins=len(P02_Cd), density=True, histtype='step',cumulative=True, color=color_dic['P02'])

plt.xlim(0,27)
plt.xlabel('Place field displacement (cm)')
plt.ylabel('Fraction of PCs')
plt.legend(['A01', 'A02', 'A03', 'A04'], loc='lower right')
plt.savefig(SaveTo2 + 'Session/%s_Centroid_SessionRemap_cumulativedis.svg' %(Saveas), dpi=300, bbox_inches="tight")
plt.show()